# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

nav = webdriver.Chrome()

nav.get('https://www.google.com.br/')

#Cotação Dólar
nav.find_element_by_xpath('html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element_by_xpath('html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print("Cotacação Dólar:", cotacao_dolar)

#Cotação Euro
nav.get('https://www.google.com.br/')
nav.find_element_by_xpath('html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print("Cotacação Euro:", cotacao_euro)

#Cotação Ouro
nav.get('https://www.melhorcambio.com/')
aba_original = nav.window_handles[0]
nav.find_element_by_xpath('/html/body/div[15]/div[2]/div/table[2]/tbody/tr[2]/td[2]/a/img').click()
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print("Cotacação Ouro:", cotacao_ouro)

nav.quit()

Cotacação Dólar: 5.2195
Cotacação Euro: 6.125344225
Cotacação Ouro: 294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [38]:
import pandas as pd

dataset = pd.read_excel('Produtos.xlsx')
display(dataset)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [39]:
dataset.loc[dataset["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
dataset.loc[dataset["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
dataset.loc[dataset["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

#Atualização Valores Preço Base Reais e Preço Final

dataset["Preço Base Reais"] = dataset["Preço Base Original"] * dataset["Cotação"]
dataset["Preço Final"] = dataset["Margem"] * dataset["Preço Base Reais"]
display(dataset)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.219500,5219.447805,1.40,7307.226927
1,Carro Renault,4500.00,Euro,6.125344,27564.049012,2.00,55128.098025
2,Notebook Dell,899.99,Dólar,5.219500,4697.497805,1.70,7985.746269
3,IPhone,799.00,Dólar,5.219500,4170.380500,1.70,7089.646850
4,Carro Fiat,3000.00,Euro,6.125344,18376.032675,1.90,34914.462082
5,Celular Xiaomi,480.48,Dólar,5.219500,2507.865360,2.00,5015.730720
6,Joia 20g,20.00,Ouro,294.120000,5882.400000,1.15,6764.760000


### Agora vamos exportar a nova base de preços atualizada

In [40]:
dataset.to_excel("Produtos_Atualizados.xlsx", index = False)